In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import traceback

In [2]:
df = pd.read_csv(r'C:\Users\user\boyu571_github\01_Kakaobank_SKKU_Research_23\eng_news_nocontent.csv')
df

,source,author,title,description,url,urlToImage,publishedAt,content,press
0,"{'id': None, 'name': 'Trendmicro.com'}",Trend Micro,REDBALDKNIGHT/BRONZE BUTLER’s Daserf Backdoor ...,"REDBALDKNIGHT, also known as BRONZE BUTLER and...",https://blog.trendmicro.com/trendlabs-security...,https://blog.trendmicro.com/trendlabs-security...,2017-11-07T12:34:44Z,by Joey Chen and MingYen Hsieh (Threat Analyst...,Trendmicro.com
1,"{'id': None, 'name': 'Computerworld'}","Lucas Mearian, Lucas Mearian",What is blockchain? The most disruptive tech i...,Blockchain is poised to change IT in much the ...,https://www.computerworld.com/article/3191077/...,https://images.techhive.com/images/article/201...,2017-11-08T01:06:00Z,Blockchain is poised to change IT in much the ...,Computerworld
2,"{'id': None, 'name': 'hackernoon.com'}",Yaoqi Jia,Demystifying Hashgraph: Benefits and Challenges,"Recently, on our Zilliqa social channels a num...",https://hackernoon.com/demystifying-hashgraph-...,NaN,2017-11-08T17:21:35Z,"Recently, on our Zilliqa social channels a num...",hackernoon.com
3,"{'id': None, 'name': 'Topix.com'}",NaN,BitTorrent inventor announces eco-friendly Bit...,A Bitcoin transaction wastes as much electrici...,http://www.topix.com/tech/p2p/2017/11/bittorre...,http://picproxy.topixcdn.com/pics/up-,2017-11-08T23:21:17Z,A Bitcoin transaction wastes as much electrici...,Topix.com
4,"{'id': None, 'name': 'Ml-implode.com'}",NaN,Bitcoin Fork Called Off: Prices Soar After Seg...,The price of a bitcoin went on a tear on Wedne...,http://ml-implode.com/staticnews/2017-11-09_Bi...,NaN,2017-11-09T00:00:00Z,The price of a bitcoin went on a tear on Wedne...,Ml-implode.com
...,...,...,...,...,...,...,...,...,...
744427,"{'id': None, 'name': 'Biztoc.com'}",u.today,"Negatively Correlates With Bitcoin (BTC), What...",Shiba Inu continues to focus on utility as its...,https://biztoc.com/x/8248d6342c37d743,https://c.biztoc.com/p/8248d6342c37d743/s.webp,2023-09-30T21:50:10Z,Shiba Inu continues to focus on utility as its...,Biztoc.com
744428,"{'id': None, 'name': 'Slashdot.org'}",feedfeeder,Industry leaders and policymakers weigh in on ...,Industry leaders and policymakers weigh in on ...,https://slashdot.org/firehose.pl?op=view&amp;i...,NaN,2023-09-30T21:53:20Z,Sign up for the Slashdot newsletter! OR check ...,Slashdot.org
744429,"{'id': None, 'name': 'Investing.com'}",U.Today,Shiba Inu (SHIB) Negatively Correlates With Bi...,Shiba Inu (SHIB) Negatively Correlates With Bi...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/LYNXNPE...,2023-09-30T22:00:22Z,U.Today - Dog-themed cryptocurrency is seeing ...,Investing.com
744430,"{'id': None, 'name': 'Ambcrypto.com'}",Benjamin Njiri,Can NEAR stay above $1?,"NEAR could be ripe for a price reversal, but t...",https://ambcrypto.com/?p=347645,https://statics.ambcrypto.com/wp-content/uploa...,2023-09-30T22:30:04Z,Disclaimer: The information presented does not...,Ambcrypto.com


In [79]:
df[df['press'] == 'Investing.com']['url'].iloc[1]

'https://www.investing.com/news/stock-market-news/asian-shares-wobbly-euro-steady-after-ecb-ends-qe-1721466'

In [80]:
Investing = df[df['press'] == 'Investing.com']
Investing

,source,author,title,description,url,urlToImage,publishedAt,content,press
19388,"{'id': None, 'name': 'Investing.com'}",Lonny Szneiberg,Investing.com Cryptocurrency — One-stop-shop f...,Live Cryptocurrency data dashboard. Overview m...,https://www.investing.com/crypto/,https://i-invdn-com.akamaized.net/investing_30...,2017-12-18T14:50:47Z,5 Key Market Questions For 2018 By Frank Holme...,Investing.com
146109,"{'id': None, 'name': 'Investing.com'}",Reuters,"Asian shares jolted by grim Chinese data, grow...","Asian shares jolted by grim Chinese data, grow...",https://www.investing.com/news/stock-market-ne...,https://i-invdn-com.akamaized.net/trkd-images/...,2018-12-14T02:57:00Z,By Hideyuki Sano and Tomo Uetake TOKYO (Reuter...,Investing.com
155588,"{'id': None, 'name': 'Investing.com'}",Reuters,Nvidia cuts fourth-quarter revenue estimate on...,Nvidia cuts fourth-quarter revenue estimate on...,https://www.investing.com/news/stock-market-ne...,https://i-invdn-com.akamaized.net/trkd-images/...,2019-01-28T16:00:00Z,By Munsif Vengattil\r\n(Reuters) - Nvidia Corp...,Investing.com
158594,"{'id': None, 'name': 'Investing.com'}",Investing.com,Stocks - Dow Sneaks a Winning Week Despite Tra...,Stocks - Dow Sneaks a Winning Week Despite Tra...,https://www.investing.com/news/stock-market-ne...,https://i-invdn-com.akamaized.net/news/LYNXNPE...,2019-02-08T22:19:00Z,Investing.com - A last-minute surge that trimm...,Investing.com
159216,"{'id': None, 'name': 'Investing.com'}",Reuters,"Wall St. opens higher on trade hopes, progress...","Wall St. opens higher on trade hopes, progress...",https://www.investing.com/news/stock-market-ne...,https://i-invdn-com.akamaized.net/trkd-images/...,2019-02-12T14:36:00Z,By Amy Caren Daniel\r\n(Reuters) - Wall Street...,Investing.com
...,...,...,...,...,...,...,...,...,...
744401,"{'id': None, 'name': 'Investing.com'}",Crypto.news,Connecticut authorities recover $3m worth of B...,Connecticut authorities recover $3m worth of B...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/Bitcoin...,2023-09-30T19:00:32Z,"Crypto.news - Westport, Connecticut police ass...",Investing.com
744407,"{'id': None, 'name': 'Investing.com'}",U.Today,"Bitcoin (BTC) Eyes Death Cross Rally, Top Expe...","Bitcoin (BTC) Eyes Death Cross Rally, Top Expe...",https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/Bitcoin...,2023-09-30T19:30:23Z,"U.Today - In a recent development, has capture...",Investing.com
744408,"{'id': None, 'name': 'Investing.com'}",Crypto.news,Bitcoin service provider Azteco secures $6m in...,Bitcoin service provider Azteco secures $6m in...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/Bitcoin...,2023-09-30T19:30:44Z,Crypto.news - Azteco raised $6 million in fund...,Investing.com
744423,"{'id': None, 'name': 'Investing.com'}",Cointelegraph,"Ether futures ETFs launching, SBF trial to beg...","Ether futures ETFs launching, SBF trial to beg...",https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/Cryptoc...,2023-09-30T21:20:17Z,Investment firm Valkyrie will start offering e...,Investing.com


In [81]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import traceback

def get_contents(yahoo):
    contents = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for url in tqdm(yahoo['url']):
        try:
            response = requests.get(url, headers=headers, timeout=60)  # 1분 timeout 설정
            soup = BeautifulSoup(response.text, "html.parser")

            # Modified this part to only get the first 'div+ article' element
            article_elements = soup.select('.articlePage')
            if article_elements:
                content = article_elements[0].get_text()
            else:
                content = ''

            contents.append(content)
        except (requests.Timeout, Exception) as e:
            if isinstance(e, requests.Timeout):
                print(f"Request timed out for URL: {url}")
            else:
                print(f"Error occurred: {e}")
                traceback.print_exc()
            contents.append('')

    return contents


In [24]:
url = 'https://www.channelnewsasia.com/business/crypto-exchange-kraken-shut-down-staking-service-pay-30-million-settlement-us-sec-3267616'
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
response = requests.get(url, headers=headers)
# soup = BeautifulSoup(response.text, "html.parser")
print(response.status_code)
# soup

200


In [25]:
# from tqdm import tqdm
# import requests
# from bs4 import BeautifulSoup

# def get_contents(yahoo):
#     contents = []

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

#     for url in tqdm(yahoo['url']):
#         try:
#             response = requests.get(url, headers=headers)
#             soup = BeautifulSoup(response.text, "html.parser")
            
#             selected_elements = soup.select('.ccn-content')
            
#             # soup.select('.ccn-content')가 빈 리스트를 반환할 경우 에러 메시지 출력
#             if not selected_elements:
#                 print(f"No elements found for URL {url}.")
#                 contents.append('')
#                 continue

#             content = []
#             for element in selected_elements:
#                 content.append(element.get_text())
#                 print(element)

#             cleaned_content = ' '.join(content)
#             contents.append(cleaned_content)
#         except Exception as e:
#             print(f"Error occurred while processing URL {url}: {e}")
#             contents.append('')

#     return contents



In [88]:
# import requests
# from bs4 import BeautifulSoup
# import re
# from tqdm import tqdm

# def get_contents(yahoo):
#     contents = []

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

#     for url in tqdm(yahoo['url']):
#         try:
#             response = requests.get(url, headers=headers)
#             soup = BeautifulSoup(response.text, "html.parser")

#             # 모든 클래스 이름 중에서 ".col-*-offset-1" 패턴을 가진 것들을 찾는다
#             classes = set()
#             for tag in soup.find_all(True, {'class': True}):
#                 for class_name in tag['class']:
#                     if re.match(r'col-\w+-offset-1', class_name):
#                         classes.add(class_name)

#             # 해당 패턴을 가진 클래스들로부터 내용을 추출한다
#             content = []
#             for class_name in classes:
#                 content.extend(soup.select(f".{class_name}"))
            
#             cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content))
#             contents.append(cleaned_content)
#         except:
#             contents.append('')

#     return contents

In [84]:
head = Investing.head()

In [88]:
# Adding the content to the DataFrame
Investing['content_full'] = get_contents(Investing)
Investing

100%|██████████| 4317/4317 [40:59<00:00,  1.76it/s]  
<ipython-input-88-4e527309f122>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Investing['content_full'] = get_contents(Investing)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
19388,"{'id': None, 'name': 'Investing.com'}",Lonny Szneiberg,Investing.com Cryptocurrency — One-stop-shop f...,Live Cryptocurrency data dashboard. Overview m...,https://www.investing.com/crypto/,https://i-invdn-com.akamaized.net/investing_30...,2017-12-18T14:50:47Z,5 Key Market Questions For 2018 By Frank Holme...,Investing.com,
146109,"{'id': None, 'name': 'Investing.com'}",Reuters,"Asian shares jolted by grim Chinese data, grow...","Asian shares jolted by grim Chinese data, grow...",https://www.investing.com/news/stock-market-ne...,https://i-invdn-com.akamaized.net/trkd-images/...,2018-12-14T02:57:00Z,By Hideyuki Sano and Tomo Uetake TOKYO (Reuter...,Investing.com,\n\n\n© Reuters. Men look at stock quotation b...
155588,"{'id': None, 'name': 'Investing.com'}",Reuters,Nvidia cuts fourth-quarter revenue estimate on...,Nvidia cuts fourth-quarter revenue estimate on...,https://www.investing.com/news/stock-market-ne...,https://i-invdn-com.akamaized.net/trkd-images/...,2019-01-28T16:00:00Z,By Munsif Vengattil\r\n(Reuters) - Nvidia Corp...,Investing.com,\n\n\n© Reuters. NVIDIA graphic cards are show...
158594,"{'id': None, 'name': 'Investing.com'}",Investing.com,Stocks - Dow Sneaks a Winning Week Despite Tra...,Stocks - Dow Sneaks a Winning Week Despite Tra...,https://www.investing.com/news/stock-market-ne...,https://i-invdn-com.akamaized.net/news/LYNXNPE...,2019-02-08T22:19:00Z,Investing.com - A last-minute surge that trimm...,Investing.com,\n\n\n© Reuters. \n\n\n\n\n \n\n\n\n\n\nUS500\...
159216,"{'id': None, 'name': 'Investing.com'}",Reuters,"Wall St. opens higher on trade hopes, progress...","Wall St. opens higher on trade hopes, progress...",https://www.investing.com/news/stock-market-ne...,https://i-invdn-com.akamaized.net/trkd-images/...,2019-02-12T14:36:00Z,By Amy Caren Daniel\r\n(Reuters) - Wall Street...,Investing.com,\n\n\n© Reuters. Trader works on the floor of ...
...,...,...,...,...,...,...,...,...,...,...
744401,"{'id': None, 'name': 'Investing.com'}",Crypto.news,Connecticut authorities recover $3m worth of B...,Connecticut authorities recover $3m worth of B...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/Bitcoin...,2023-09-30T19:00:32Z,"Crypto.news - Westport, Connecticut police ass...",Investing.com,\n\n\n© Reuters Connecticut authorities recove...
744407,"{'id': None, 'name': 'Investing.com'}",U.Today,"Bitcoin (BTC) Eyes Death Cross Rally, Top Expe...","Bitcoin (BTC) Eyes Death Cross Rally, Top Expe...",https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/Bitcoin...,2023-09-30T19:30:23Z,"U.Today - In a recent development, has capture...",Investing.com,\n\n\n© Igor Faun Bitcoin (BTC) Eyes Death Cro...
744408,"{'id': None, 'name': 'Investing.com'}",Crypto.news,Bitcoin service provider Azteco secures $6m in...,Bitcoin service provider Azteco secures $6m in...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/Bitcoin...,2023-09-30T19:30:44Z,Crypto.news - Azteco raised $6 million in fund...,Investing.com,\n\n\nBitcoin service provider Azteco secures ...
744423,"{'id': None, 'name': 'Investing.com'}",Cointelegraph,"Ether futures ETFs launching, SBF trial to beg...","Ether futures ETFs launching, SBF trial to beg...",https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/Cryptoc...,2023-09-30T21:20:17Z,Investment firm Valkyrie will start offering e...,Investing.com,"\n\n\nEther futures ETFs launching, SBF trial ..."


In [87]:
head['url'].iloc[0]

'https://www.investing.com/crypto/'

In [91]:
empty_rows['url'].iloc[0]

'https://www.investing.com/crypto/'

In [26]:
head['url'].iloc[1]

'https://www.channelnewsasia.com/singapore/us28-million-cryptocurrency-withdrawn-vang-shuiming-remanded-io-affidavit-3808541'

In [29]:
# with1 = PRNewswire[PRNewswire['content_full'] != '']
without['url'].iloc[10]

'https://mediablog.prnewswire.com/2020/07/15/9-unique-finance-news-sites-that-are-worth-investing-your-time-in/'

In [90]:
empty_rows = Investing[Investing['content_full'] == '']

In [88]:
with1 =  Bloomberg[Bloomberg['content_full'] != '']
empty_rows = Bloomberg[Bloomberg['content_full'] == '']

In [95]:
# Bloomberg[Bloomberg['content_full'] != '']
len(with1)

15

In [55]:
empty_rows = CryptoSlate[CryptoSlate['content_full'] == '']
empty_rows['url'].iloc[20]
# print(len(CryptoSlate[CryptoSlate['content_full'] == '']))

'https://cryptoslate.com/research-the-data-behind-bitcoins-energy-fud-bitcoin-mining-consumes-more-electricity-than/'

In [32]:
CNA = pd.concat([CNA, empty_rows, empty_rows], ignore_index=True)

CNA

,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
0,"{'id': None, 'name': 'CNA'}",NaN,Nvidia Corp reports 54.6 percent rise in quart...,Nvidia Corp reported a 54.6 percent rise in q...,http://www.channelnewsasia.com/news/business/n...,http://www.channelnewsasia.com/image/9392908/1...,2017-11-09T21:40:14Z,REUTERS: Nvidia Corp reported a 54.6 percent r...,CNA,\nBest News Website or Mobile Service\n\nWAN-I...
1,"{'id': None, 'name': 'CNA'}",NaN,Bitcoin slides by over US$1000 in less than 48...,"Bitcoin dropped below US$7,000 on Friday to tr...",http://www.channelnewsasia.com/news/business/b...,http://www.channelnewsasia.com/image/9395936/1...,2017-11-10T12:35:15Z,"LONDON: Bitcoin dropped below US$7,000 on Frid...",CNA,\nBest News Website or Mobile Service\n\nWAN-I...
2,"{'id': None, 'name': 'CNA'}",NaN,"Bitcoin slides by over US$1,000 in less than 4...","Bitcoin dropped below US$7,000 on Friday to tr...",http://www.channelnewsasia.com/news/technology...,http://www.channelnewsasia.com/image/9395940/1...,2017-11-10T17:30:30Z,"LONDON: Bitcoin dropped below US$7,000 on Frid...",CNA,\nBest News Website or Mobile Service\n\nWAN-I...
3,"{'id': None, 'name': 'CNA'}",Janice Lim,More Singaporeans riding on cryptocurrency wav...,SINGAPORE: It looks like a simple gaming compu...,http://www.channelnewsasia.com/news/business/m...,http://www.channelnewsasia.com/image/9399916/1...,2017-11-13T00:16:43Z,SINGAPORE: It looks like a simple gaming compu...,CNA,\nBest News Website or Mobile Service\n\nWAN-I...
4,"{'id': None, 'name': 'CNA'}",NaN,Asia shares down on caution over US tax reform...,Asian shares stepped back in cautious early tr...,http://www.channelnewsasia.com/news/business/a...,http://www.channelnewsasia.com/image/9400328/1...,2017-11-13T01:10:10Z,TOKYO: Asian shares stepped back in cautious e...,CNA,\nBest News Website or Mobile Service\n\nWAN-I...
...,...,...,...,...,...,...,...,...,...,...
4538,"{'id': None, 'name': 'CNA'}",NaN,Carl Icahn holds short position in GameStop - ...,Activist investor Carl Icahn has a sizeable sh...,https://www.channelnewsasia.com/business/carl-...,https://onecms-res.cloudinary.com/image/upload...,2022-11-21T23:51:27Z,Activist investor Carl Icahn has a sizeable sh...,CNA,\nBest News Website or Mobile Service\n\nWAN-I...
4539,"{'id': None, 'name': 'CNA'}",NaN,Global rules leave crypto firms with no place ...,LONDON : Globally agreed rules leave crypto fi...,https://www.channelnewsasia.com/business/globa...,https://onecms-res.cloudinary.com/image/upload...,2023-07-17T07:07:11Z,LONDON : Globally agreed rules leave crypto fi...,CNA,\nBest News Website or Mobile Service\n\nWAN-I...
4540,"{'id': None, 'name': 'CNA'}",NaN,Ripple Labs interested in bankrupt crypto lend...,:San Francisco-based blockchain payments compa...,https://www.channelnewsasia.com/business/rippl...,https://onecms-res.cloudinary.com/image/upload...,2022-08-10T14:52:48Z,:San Francisco-based blockchain payments compa...,CNA,\nBest News Website or Mobile Service\n\nWAN-I...
4541,"{'id': None, 'name': 'CNA'}",NaN,Carl Icahn holds short position in GameStop - ...,Activist investor Carl Icahn has a sizeable sh...,https://www.channelnewsasia.com/business/carl-...,https://onecms-res.cloudinary.com/image/upload...,2022-11-21T23:51:27Z,Activist investor Carl Icahn has a sizeable sh...,CNA,\nBest News Website or Mobile Service\n\nWAN-I...


In [92]:
Investing.to_csv(r'C:\Users\user\boyu571_github\01_Kakaobank_SKKU_Research_23\Investing_30.csv', encoding='utf-8-sig')

In [ ]:
#article-13715 > div.post__content-wrapper > div.post-content